## Importing libs

In [2]:
import time
import numpy as np
import pandas as pd
import pickle

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import json

### Set up selenium browser

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('headless')    
service = Service('./chromedriver')
driver = webdriver.Chrome(service=service, options=options)

## Web scraping TASS

In [4]:
def get_content(driver: webdriver, api: str) -> json:
    '''Open a link by selenium webdriver, scrap the page and returns result with proposed news in json format'''
    driver.get(api) # открываем ссылку
    time.sleep(3) # даем прогрузиться
    
    page_source = driver.page_source # берем страницу
    soup = BeautifulSoup(page_source, 'html.parser') # парсим
    data = json.loads(soup.body.text) # достаем текст и превращаем в формат json
    
    return data


def update_news(news: list, data: json):
    '''Download news from Tass ural (/ural-news) and add to the existing list of news
    '''
    new_news = [] # список всех распарсенных новостей
    for item in data['result']: # идем по каждой новости
        cur_news = {} 
        cur_news['composite_id'] = item['composite_id']
        cur_news['title'] = item['title']
        cur_news['subtitle'] = item['subtitle']
        cur_news['lead'] = item['lead']
        cur_news['url'] = 'https://tass.ru' + item['url']
        cur_news['rubrics'] = []
        for rubric in item['rubrics']:
            cur_news['rubrics'].append(rubric['name'])
        cur_news['tags'] = []
        for tag in item['tags']:
            cur_news['tags'].append(tag['name'])
        cur_news['es_updated_dt'] = item['es_updated_dt']
        cur_news['published_date'] = item['es_updated_dt'].split('T')[0]
        new_news.append(cur_news)
        
    news.extend(new_news) # расширяем существующий список новыми новостями
    pass

In [5]:
# загружаем новости со стартовой страницы
api = 'https://tass.ru/tbp/api/v1/content?limit=18&offset=0&lang=ru&rubrics=/ural-news&sort=-es_updated_dt'
data = get_content(driver, api)

In [6]:
data['result'][0] # так выглядит новость

{'id': 14976661,
 'composite_id': '14976661',
 'type': 'news',
 'status': 'published',
 'title': 'Грозы с сильным ветром и жаркая погода ожидаются в регионах Урала в ближайшие три дня',
 'subtitle': '',
 'lead': 'Синоптики прогнозируют грозы с сильным ветром и дожди',
 'url': '/obschestvo/14976661',
 'lang': 'ru',
 'urgency': 'normal',
 'main_media': {'gallery': None, 'image': None, 'video': None},
 'main_rubric_id': 24,
 'rubrics': [{'id': 24,
   'name': 'Общество',
   'url': '/obschestvo',
   'type': 'rubric'},
  {'id': 4731,
   'name': 'Новости Урала',
   'url': '/ural-news',
   'type': 'rubric'}],
 'tags': [{'id': 3929, 'name': 'Россия', 'url': '/rossiya'},
  {'id': 8974,
   'name': 'Свердловская область',
   'url': '/sverdlovskaya-oblast'}],
 'published_dt': '2022-06-20T13:53:40',
 'publish_updated_dt': '2022-06-20T13:53:40',
 'updated_dt': '2022-06-20T13:53:40',
 'es_updated_dt': '2022-06-20T13:53:40.775132',
 'meta_title': 'Грозы с сильным ветром и жаркая погода ожидаются в реги

In [8]:
news[0] # так выглядит каждая новость после парсинга

{'composite_id': '14976661',
 'title': 'Грозы с сильным ветром и жаркая погода ожидаются в регионах Урала в ближайшие три дня',
 'subtitle': '',
 'lead': 'Синоптики прогнозируют грозы с сильным ветром и дожди',
 'url': 'https://tass.ru/obschestvo/14976661',
 'rubrics': ['Общество', 'Новости Урала'],
 'tags': ['Россия', 'Свердловская область'],
 'es_updated_dt': '2022-06-20T13:53:40.775132',
 'published_date': '2022-06-20'}

## Downloading past news

In [10]:
with open('tass', 'rb') as f:
    news = pickle.load(f)

In [26]:
i = 0
while True:
    time.sleep(3)
    i += 1
    last_es_updated_dt = news[-1]['es_updated_dt']
    iter_api = f'https://tass.ru/tbp/api/v1/content?limit=15&last_es_updated_dt={last_es_updated_dt}&lang=ru&rubrics=/ural-news&sort=-es_updated_dt'
    data = get_content(driver, iter_api)
    update_news(news, data)
    if i == 100:
        print('Reached 100!')
        i = 0
        with open('tass', 'wb') as f:
            pickle.dump(news, f)

Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!


KeyboardInterrupt: 

In [ ]:
driver.close()

In [18]:
with open('tass', 'wb') as f:
    pickle.dump(news, f)

In [27]:
len(news)

34321

In [28]:
news[-1]

{'composite_id': '697849',
 'title': 'Лосенок, которого спасли на Урале, первые в жизни морозы перезимует в конюшне',
 'subtitle': '',
 'lead': 'Подкидыша выходили свердловские школьники',
 'url': 'https://tass.ru/ural-news/697849',
 'rubrics': ['Новости Урала'],
 'tags': [],
 'es_updated_dt': '2013-10-29T10:54:39.461413',
 'published_date': '2013-10-29'}